In [2]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx

In [3]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [4]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)

In [5]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [6]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [7]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [8]:
nj_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

,GEOID20,District
0,34033020001,2
1,34033001501,2
2,34033042009,2
3,34033000703,2
4,34033042001,2
...,...,...
6356,34039045302,7
6357,34039045303,7
6358,34039045404,7
6359,34039045801,7


### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [9]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [10]:
shpfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data 

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv. 

In [11]:
nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)

In [12]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(12, nj_current_assignment, nj_contiguity)))

District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True


In [13]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    

C:\Users\aqila\AppData\Local\Temp\ipykernel_9248\1799164020.py:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if shape(boundaries[i]).type == 'Polygon':
C:\Users\aqila\AppData\Local\Temp\ipykernel_9248\1799164020.py:7: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif shape(boundaries[i]).type == 'MultiPolygon':


D1 PP : 0.41768102211569347
D1 BR : 0.45075813446417157
D2 PP : 0.2632665176502347
D2 BR : 0.38278056561756263
D3 PP : 0.2280937682959879
D3 BR : 0.38134920642809134
D4 PP : 0.24812480573284196
D4 BR : 0.5390173747196018
D5 PP : 0.2410116694999733
D5 BR : 0.36320426268176653
D6 PP : 0.14677124653853732
D6 BR : 0.32853496486220907
D7 PP : 0.20246375771704353
D7 BR : 0.44049249082841035
D8 PP : 0.11227347882175574
D8 BR : 0.36670629634952656
D9 PP : 0.1683197710884751
D9 BR : 0.29705227593212374
D10 PP : 0.12061263370064262
D10 BR : 0.34528827672774703
D11 PP : 0.22236600778446886
D11 BR : 0.5557086439792166
D12 PP : 0.1620092442171186
D12 BR : 0.38520439164401626


In [14]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

{1: 775340, 2: 778354, 3: 778489, 4: 767834, 5: 774454, 6: 778516, 7: 785173, 8: 800074, 9: 766863, 10: 746178, 11: 769523, 12: 768196}


# A simple geographical  redistricting strategy

We can create a simple geopgraphical map, like we did for NH. In this case, we have 12 districts, so let's splitting the district in half North/South, and in 6th  East/West. 
New Hampshire's bounding box is (-75.559614,38.928519,-73.893979,41.357423) (https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/)
So let's start by splitting the state approximately though the middle longitude (-74.72) : everything west of longitude -71.583934 is in odd Districts, everything east is in even Districts. We will use the precinct centroids to assign them. Then we will divide each half per latitude on the ranges  (38.92, 39.3, 39.7, 40.1, 40.5,40.9,41.35)
Import the Map to DRA to look at it.

In [15]:
nj_longlat_assignment = nj_current_assignment.copy()
nj_longlat_assignment['District'] = 0
for index, row in nj_longlat_assignment.iterrows():
    try:
        if shape(nj_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
            centroid = Polygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        elif shape(nj_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':
            centroid = MultiPolygon(shape(nj_precinct_boundaries[row['GEOID20']])).centroid
        else:
            print(shape(nj_precinct_boundaries[row['GEOID20']]).type)
            pass
        if centroid.x <= -74.72:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 1
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 3
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 5
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 7
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 9
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 11
        else:
            if centroid.y <= 39.3:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 2
            elif centroid.y <= 39.7:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 4
            elif centroid.y <= 40.1:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 6
            elif centroid.y <= 40.5:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 8
            elif centroid.y <= 40.9:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 10
            else:
                nj_longlat_assignment.iloc[index,nj_longlat_assignment.columns.get_loc('District')] = 12
    except KeyError: 
        pass
#print(nh_longitude_assignment)
nj_longlat_assignment.to_csv('Recitation maps/nj_longlat_map.csv',index=False)

C:\Users\aqila\AppData\Local\Temp\ipykernel_9248\3774939677.py:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if shape(nj_precinct_boundaries[row['GEOID20']]).type == 'Polygon':
C:\Users\aqila\AppData\Local\Temp\ipykernel_9248\3774939677.py:7: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif shape(nj_precinct_boundaries[row['GEOID20']]).type == 'MultiPolygon':


In [16]:
#Compactness of this Longlat assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_longlat_assignment,nj_precinct_boundaries))))
    

C:\Users\aqila\AppData\Local\Temp\ipykernel_9248\1799164020.py:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if shape(boundaries[i]).type == 'Polygon':


D1 PP : 0.24444322502174795
D1 BR : 0.39921574154934536
D2 PP : 0.21732375139633123
D2 BR : 0.3033805826444085


C:\Users\aqila\AppData\Local\Temp\ipykernel_9248\1799164020.py:7: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif shape(boundaries[i]).type == 'MultiPolygon':


D3 PP : 0.3012927570664991
D3 BR : 0.7281613087250183
D4 PP : 0.3019375759906228
D4 BR : 0.6388831361756292
D5 PP : 0.29897783517755583
D5 BR : 0.5136294328843819
D6 PP : 0.22596255307162377
D6 BR : 0.5362987383285662
D7 PP : 0.21866681223528045
D7 BR : 0.4337413755961903
D8 PP : 0.3513739185552307
D8 BR : 0.82797734690796
D9 PP : 0.37261194943216025
D9 BR : 0.7269867982048943
D10 PP : 0.28764401412152446
D10 BR : 0.6900841778845721
D11 PP : 0.34638282192061104
D11 BR : 0.47137706735305646
D12 PP : 0.318631303145351
D12 BR : 0.5828294919378438


In [17]:
# District Population of this longlat assignment
print(getDistrictPopulations(nj_longlat_assignment,nj_precinct_data, 12))


{1: 80406, 2: 22492, 3: 317218, 4: 274411, 5: 1124427, 6: 563593, 7: 244052, 8: 1471840, 9: 230117, 10: 3848094, 11: 61026, 12: 1051318}


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

In [18]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


In [19]:
def flip_precincts(current_map, district_from, district_to, num_flip, contiguity_data, precinct_data):
    border_precincts = []
    current_list = current_map[current_map['District']==district_from]['GEOID20']
    for precinct in current_list:
            neighbors = ast.literal_eval(
                contiguity_data[contiguity_data['Precinct'] == precinct]['Neighbors'].values.tolist()[0])
            for neighbor in neighbors:
                if current_map.loc[current_map['GEOID20'] == neighbor, 'District'].values.tolist()[
                    0] == district_to:
                    border_precincts.append(precinct)
                    
    flipped_precincts = np.random.choice(border_precincts, num_flip)
    for flip in flipped_precincts:
        current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_to
        # check if we broke contiguity and revert the flip if we did
        if (not isDistrictContiguous(district_from, current_map, contiguity_data) or not isDistrictContiguous(district_to, current_map, contiguity_data)):
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Contiguity broken " + flip)
        else:
            print("Went through " + flip)

# Fair Map

## From Existing Map - Flip Step
random based strategy similar to one in recitation_redistricting notebook, flips 

In [20]:
def populationDifference(assignment,data_file, district_from, district_to):
    populations = getDistrictPopulations(assignment, data_file, 12)
    pop_from = populations[district_from]
    pop_to = populations[district_to]
    difference = abs(pop_from - pop_to) / ((pop_from + pop_to) / 2)
    return difference

In [21]:
def get_party_data(precinct_data, district):

  party_data = {}
  district_df = precinct_data[precinct_data['District'] == district]  

  total_votes = sum(district_df['Total_2020_Pres'])
  dem_votes = sum(district_df['Dem_2020_Pres']) 
  rep_votes = sum(district_df['Rep_2020_Pres'])

  party_data['total'] = total_votes
  party_data['dem'] = dem_votes
  party_data['rep'] = rep_votes

  return party_data

In [44]:
def fair_flip(current_map, district_from, district_to, num_flip, contiguity_data, precinct_data):
    border_precincts = []
    current_list = current_map[current_map['District']==district_from]['GEOID20']
    for precinct in current_list:
            neighbors = ast.literal_eval(
                contiguity_data[contiguity_data['Precinct'] == precinct]['Neighbors'].values.tolist()[0])
            for neighbor in neighbors:
                if current_map.loc[current_map['GEOID20'] == neighbor, 'District'].values.tolist()[
                    0] == district_to:
                    border_precincts.append(precinct)
                    
    flipped_precincts = np.random.choice(border_precincts, num_flip)
    for flip in flipped_precincts:
        current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_to
        # check if we broke contiguity and revert the flip if we did
        if (not isDistrictContiguous(district_from, current_map, contiguity_data) or not isDistrictContiguous(district_to, current_map, contiguity_data)):
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Contiguity broken " + flip)
        elif (populationDifference(current_map, nj_precinct_data, district_from, district_to)) > 0.05:
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Population deviation too high " + flip)

        from_data = get_party_data(precinct_data, district_from)
        to_data = get_party_data(precinct_data, district_to)
        total_votes = from_data['total'] + to_data['total']
        dem_percentage = (from_data['dem'] + to_data['dem']) / total_votes
        rep_percentage = (from_data['rep'] + to_data['rep']) / total_votes
        imbalance = abs(dem_percentage - rep_percentage)
        if (imbalance > 0.25):
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Party imbalance too high " + flip)


In [23]:
nj_flipstep_assignment = nj_current_assignment.copy()

# Simplistic Approach

fair_flip(nj_flipstep_assignment, 1, 2, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 2, 1, 8, nj_contiguity, nj_precinct_data)
print("District 1 and 2 Flipped")

fair_flip(nj_flipstep_assignment, 2, 3, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 3, 2, 8, nj_contiguity, nj_precinct_data)
print("District 2 and 3 Flipped")

fair_flip(nj_flipstep_assignment, 2, 4, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 4, 2, 8, nj_contiguity, nj_precinct_data)
print("District 2 and 4 Flipped")

fair_flip(nj_flipstep_assignment, 2, 6, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 6, 2, 8, nj_contiguity, nj_precinct_data)
print("District 2 and 6 Flipped")

fair_flip(nj_flipstep_assignment, 1, 3, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 3, 1, 8, nj_contiguity, nj_precinct_data)
print("District 1 and 3 Flipped")

fair_flip(nj_flipstep_assignment, 3, 4, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 4, 3, 8, nj_contiguity, nj_precinct_data)
print("District 3 and 4 Flipped")

fair_flip(nj_flipstep_assignment, 3, 12, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 12, 3, 8, nj_contiguity, nj_precinct_data)
print("District 3 and 12 Flipped")

fair_flip(nj_flipstep_assignment, 3, 6, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 6, 3, 8, nj_contiguity, nj_precinct_data)
print("District 3 and 6 Flipped")

fair_flip(nj_flipstep_assignment, 4, 6, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 6, 4, 8, nj_contiguity, nj_precinct_data)
print("District 4 and 6 Flipped")

fair_flip(nj_flipstep_assignment, 12, 7, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 7, 12, 8, nj_contiguity, nj_precinct_data)
print("District 12 and 7 Flipped")

fair_flip(nj_flipstep_assignment, 12, 6, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 6, 12, 8, nj_contiguity, nj_precinct_data)
print("District 12 and 6 Flipped")

fair_flip(nj_flipstep_assignment, 7, 11, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 11, 7, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 7 Flipped")

fair_flip(nj_flipstep_assignment, 9, 11, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 11, 11, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 9 Flipped")

fair_flip(nj_flipstep_assignment, 11, 5, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 5, 11, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 5 Flipped")

fair_flip(nj_flipstep_assignment, 11, 10, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 10, 11, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 10 Flipped")

fair_flip(nj_flipstep_assignment, 7, 5, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 5, 7, 8, nj_contiguity, nj_precinct_data)
print("District 7 and 5 Flipped")

fair_flip(nj_flipstep_assignment, 9, 5, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 5, 9, 8, nj_contiguity, nj_precinct_data)
print("District 9 and 5 Flipped")

fair_flip(nj_flipstep_assignment, 9, 8, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 8, 9, 8, nj_contiguity, nj_precinct_data)
print("District 9 and 8 Flipped")

fair_flip(nj_flipstep_assignment, 10, 8, 8, nj_contiguity, nj_precinct_data)
fair_flip(nj_flipstep_assignment, 8, 10, 8, nj_contiguity, nj_precinct_data)
print("District 10 and 8 Flipped")

for district in range(1, 13):
  contiguous = isDistrictContiguous(district, nj_flipstep_assignment, nj_contiguity)
  print("District", district, "Contiguous?", contiguous)

nj_flipstep_assignment.to_csv('Recitation maps/fairmap.csv',index=False)

District 1 and 2 Flipped
Contiguity broken 34005005001
Contiguity broken 34005185001
District 2 and 3 Flipped
District 2 and 4 Flipped
District 2 and 6 Flipped
Contiguity broken 34007135009
Contiguity broken 34005065020
District 1 and 3 Flipped
District 3 and 4 Flipped
District 3 and 12 Flipped
Contiguity broken 34025090002
Contiguity broken 34025090002
Contiguity broken 34025090002
District 3 and 6 Flipped
Contiguity broken 34025035002
Contiguity broken 34025035001
District 4 and 6 Flipped
District 12 and 7 Flipped
Party imbalance too high 34035040546
Party imbalance too high 34023055010
Party imbalance too high 34023020013
Party imbalance too high 34035040545
Party imbalance too high 34039060308
Party imbalance too high 34023075012
Party imbalance too high 34023077401
Party imbalance too high 34023075003
Party imbalance too high 34023085301
Party imbalance too high 34023095002
Party imbalance too high 34023085305
Party imbalance too high 34023085101
Party imbalance too high 340230853

# Unfair Map

In [48]:
def gerrymandered_flip(current_map, district_from, district_to, num_flip, contiguity_data, precinct_data):
    border_precincts = []
    current_list = current_map[current_map['District']==district_from]['GEOID20']
    for precinct in current_list:
            neighbors = ast.literal_eval(
                contiguity_data[contiguity_data['Precinct'] == precinct]['Neighbors'].values.tolist()[0])
            for neighbor in neighbors:
                if current_map.loc[current_map['GEOID20'] == neighbor, 'District'].values.tolist()[
                    0] == district_to:
                    border_precincts.append(precinct)
                    
    # border_precincts = sorted(border_precincts, key=lambda x: precinct_data.loc[precinct_data['GEOID20'] == x, 'Dem_2020_Pres'].values[0], reverse=True)                
    border_precincts = sorted(border_precincts, key=lambda x: (
         precinct_data.loc[precinct_data['GEOID20'] == x, 'Dem_2020_Pres'].values[0],
         precinct_data.loc[precinct_data['GEOID20'] == x, 'Rep_2020_Pres'].values[0]
    ), reverse=True)

    flipped_precincts = np.random.choice(border_precincts, num_flip)
    for flip in flipped_precincts:
        current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_to
        # check if we broke contiguity and revert the flip if we did
        if (not isDistrictContiguous(district_from, current_map, contiguity_data) or not isDistrictContiguous(district_to, current_map, contiguity_data)):
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Contiguity broken " + flip)
        elif (populationDifference(current_map, nj_precinct_data, district_from, district_to)) > 0.05:
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Population deviation too high " + flip)

        # Print columns of the DataFrame
        # print(nj_precinct_data.columns)
        
        # Check if the flip makes the district more Republican
        before_rep_votes = nj_precinct_data[nj_precinct_data['District'] == district_to]['Rep_2020_Pres'].sum()
        after_rep_votes = nj_precinct_data[nj_precinct_data['District'] == district_to]['Rep_2020_Pres'].sum()


        if after_rep_votes < before_rep_votes:
            current_map.loc[current_map['GEOID20'] == flip, 'District'] = district_from
            print("Flipping would make the district less Republican " + flip)

In [49]:
nj_unfair_map = nj_current_assignment.copy()

gerrymandered_flip(nj_unfair_map, 1, 2, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 2, 1, 8, nj_contiguity, nj_precinct_data)
print("District 1 and 2 Flipped")

gerrymandered_flip(nj_unfair_map, 2, 3, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 3, 2, 8, nj_contiguity, nj_precinct_data)
print("District 2 and 3 Flipped")

gerrymandered_flip(nj_unfair_map, 2, 4, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 4, 2, 8, nj_contiguity, nj_precinct_data)
print("District 2 and 4 Flipped")

gerrymandered_flip(nj_unfair_map, 2, 6, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 6, 2, 8, nj_contiguity, nj_precinct_data)
print("District 2 and 6 Flipped")

gerrymandered_flip(nj_unfair_map, 1, 3, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 3, 1, 8, nj_contiguity, nj_precinct_data)
print("District 1 and 3 Flipped")

gerrymandered_flip(nj_unfair_map, 3, 4, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 4, 3, 8, nj_contiguity, nj_precinct_data)
print("District 3 and 4 Flipped")

gerrymandered_flip(nj_unfair_map, 3, 12, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 12, 3, 8, nj_contiguity, nj_precinct_data)
print("District 3 and 12 Flipped")

gerrymandered_flip(nj_unfair_map, 3, 6, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 6, 3, 8, nj_contiguity, nj_precinct_data)
print("District 3 and 6 Flipped")

gerrymandered_flip(nj_unfair_map, 4, 6, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 6, 4, 8, nj_contiguity, nj_precinct_data)
print("District 4 and 6 Flipped")

gerrymandered_flip(nj_unfair_map, 12, 7, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 7, 12, 8, nj_contiguity, nj_precinct_data)
print("District 12 and 7 Flipped")

gerrymandered_flip(nj_unfair_map, 12, 6, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 6, 12, 8, nj_contiguity, nj_precinct_data)
print("District 12 and 6 Flipped")

gerrymandered_flip(nj_unfair_map, 7, 11, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 11, 7, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 7 Flipped")

gerrymandered_flip(nj_flipstep_assignment, 9, 11, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_flipstep_assignment, 11, 11, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 9 Flipped")

gerrymandered_flip(nj_unfair_map, 11, 5, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 5, 11, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 5 Flipped")

gerrymandered_flip(nj_unfair_map, 11, 10, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 10, 11, 8, nj_contiguity, nj_precinct_data)
print("District 11 and 10 Flipped")

gerrymandered_flip(nj_unfair_map, 7, 5, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 5, 7, 8, nj_contiguity, nj_precinct_data)
print("District 7 and 5 Flipped")

gerrymandered_flip(nj_unfair_map, 9, 5, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 5, 9, 8, nj_contiguity, nj_precinct_data)
print("District 9 and 5 Flipped")

gerrymandered_flip(nj_unfair_map, 9, 8, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 8, 9, 8, nj_contiguity, nj_precinct_data)
print("District 9 and 8 Flipped")

gerrymandered_flip(nj_unfair_map, 10, 8, 8, nj_contiguity, nj_precinct_data)
gerrymandered_flip(nj_unfair_map, 8, 10, 8, nj_contiguity, nj_precinct_data)
print("District 10 and 8 Flipped")

for district in range(1, 13):
  contiguous = isDistrictContiguous(district, nj_unfair_map, nj_contiguity)
  print("District", district, "Contiguous?", contiguous)

nj_unfair_map.to_csv('Recitation maps/unfairTest.csv',index=False)

District 1 and 2 Flipped
Contiguity broken 34005005001
Contiguity broken 34005005001
Contiguity broken 34005195001
District 2 and 3 Flipped
District 2 and 4 Flipped
District 2 and 6 Flipped
Contiguity broken 34007135009
Contiguity broken 34005140002
Contiguity broken 34005065020
District 1 and 3 Flipped
